In [1]:
import numpy as np
from uav_boxes import Village
from seeding_utils import point_near_regions, vis_reg, point_in_regions
from scipy.sparse import lil_matrix
import numpy as np
from tqdm import tqdm
from region_generation import generate_regions_multi_threading

/home/peter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
world = Village()
village_side = 29
world.build(village_height=3, village_side=village_side, building_every=5, density=0.15)
print(len(world.obstacles))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
273


In [3]:
N = 1
seed = 0
alpha = 0.05
eps = 0.2

In [4]:
def sample_cfree_handle(n, m, regions=None):
    points = np.zeros((n,3))
    m = m*100
    if regions is None: regions = []		
    for i in range(n):
        bt_tries = 0
        while bt_tries<m:
            point = world.sample_cfree(1)[0]
            #point = world.sample_cfree(1)[0]
            if point_near_regions(point, regions, tries = 100, eps = 0.1):
                bt_tries+=1
            else:
                break
        # if bt_tries == m:
        #     return points, True
        points[i] = point
    return points, False

def estimate_coverage(regions):
    n_s = 5000
    samples = world.sample_cfree(n_s)
    in_s = 0
    for s in samples:
        if point_in_regions(s, regions):
            in_s+=1
    return (1.0*in_s)/n_s

def vgraph_builder(points, regions, region_vis_obstacles=True):
    n = len(points)
    adj_mat = lil_matrix((n,n))
    for i in tqdm(range(n)):
        point = points[i, :]
        for j in range(len(points[:i])):
            other = points[j]
            if region_vis_obstacles:
                if vis_reg(point, other, world, regions):
                    adj_mat[i,j] = adj_mat[j,i] = 1
            else:
                if vis_reg(point, other, world, []):
                    adj_mat[i,j] = adj_mat[j,i] = 1
    return adj_mat.toarray()

def iris_w_obstacles(points, region_obstacles, old_regions = None, use_region_obstacles = True):
    if N>1:
        #+ region_obstacles
        obstacles = [r for r in world.obstacles]
        if use_region_obstacles:
            obstacles += region_obstacles
        regions, _, is_full = generate_regions_multi_threading(points, obstacles, world.iris_domain, estimate_coverage, coverage_threshold=1-eps, old_regs = old_regions)
    else:
        #if N=1 coverage estimate happens at every step
        obstacles = [r for r in world.obstacles]
        if use_region_obstacles:
            obstacles += region_obstacles
        regions, _, _ = generate_regions_multi_threading(points, obstacles, world.iris_domain, estimate_coverage, coverage_threshold=1-eps, old_regs=old_regions)
        is_full = 1-eps <= estimate_coverage(old_regions+regions)
    return regions, is_full

In [22]:
from vislogging import Logger3D
from visibility_seeding import VisSeeder
from region_generation import generate_regions_ellipses_multi_threading
# # Capture the image
# image_data = w.meshcat.StaticHtml()
from time import strftime, gmtime
N = 1
logger = Logger3D(world, f"village_naive_{village_side}_"+strftime("%m_%d_%H_%M_%S_", gmtime())+"_", seed, N, alpha, eps, estimate_coverage)
VS = VisSeeder(N = N,
            alpha = alpha,
            eps = eps,
            max_iterations = 300,
            sample_cfree = sample_cfree_handle,
            build_vgraph = vgraph_builder,
            iris_w_obstacles = iris_w_obstacles,
            verbose = True,
            logger = logger
            )

regions = VS.run()

logdir created
[00:24:07] [VisSeeder] GuardInsertion attempts M: 13
[00:24:07] [VisSeeder] 0.95 probability that unseen region is less than 20.0 '%' of Cfree 


100%|██████████| 1/1 [00:00<00:00, 25115.59it/s]

[00:24:07] [VisSeeder] Found  1  hidden points
[00:24:07]  1 / 1



100%|██████████| 1/1 [00:00<00:00, 23563.51it/s]


[00:24:17] [VisSeeder] Found  1  hidden points
[00:24:17]  1 / 1


...

[00:55:11] [VisSeeder] Found  1  hidden points
[00:55:11]  1 / 1


100%|██████████| 1/1 [00:00<00:00, 30615.36it/s]


[00:55:26] [VisSeeder] Found  1  hidden points
[00:55:26]  1 / 1


100%|██████████| 1/1 [00:00<00:00, 30615.36it/s]


[00:55:50] [VisSeeder] Found  1  hidden points
[00:55:50]  1 / 1


In [5]:
from vislogging import LoggerClique3D
from visibility_clique_decomposition import VisCliqueDecomp
from region_generation import generate_regions_ellipses_multi_threading
from time import strftime,gmtime
from functools import partial

approach = 0
N = 100
ap_names = ['redu', 'greedy', 'nx']
loggerccd = LoggerClique3D(world, f"village_{village_side}_{ap_names[approach]}_"+strftime("%m_%d_%H_%M_%S_", gmtime())+"_", seed, N, alpha, eps, estimate_coverage)
def iris_ellipse_w_obstacles_handle(points, ellipsoids, old_regions = None):
    if N>=1:
        #+ region_obstacles
        obstacles = [r for r in world.obstacles]
        regions, _, is_full = generate_regions_ellipses_multi_threading(points, ellipsoids, obstacles, world.iris_domain, estimate_coverage, coverage_threshold=1-eps, old_regs = old_regions, maxiters=3)
    return regions, is_full
VCD = VisCliqueDecomp(  N = N,
                        eps = eps,
                        max_iterations=10,
                        sample_cfree=sample_cfree_handle,
                        col_handle=world.col_handle,
                        build_vgraph=partial(vgraph_builder, regions = [], region_vis_obstacles = False),
                        iris_w_obstacles= iris_ellipse_w_obstacles_handle,
                        verbose=True,
                        logger=loggerccd, 
                        approach = approach
                    )

VCD.run()

logdir created
[13:19:06] [VisCliqueDecomp] Attempting to cover 80.0 '%' of Cfree 


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:27<00:00,  3.65it/s]


VERTEX CLIQUE COVERING PROBLEM (CCP) SOLVER
 version 0.2
 Finding clique coverings using our heuristic...
 GIS:
 Run 1:
 The algorithm found a an independent set of size 18.
 IG-GCC:
 Run 72:
 Time\t\tVCC\tIS
 0.000396\t27\t21
 0.000415\t26\t22
 0.000425\t24\t22
 0.000511\t23\t22
  [iterations: 3891203][independent set: 19]
 
 The algorithm needed 20 cliques.
 
 On average, it took 0 seconds and 0 iterations. Success rate: 0 / 1.
 Final results: IG-GCC: 20.00, BRE: 0.00, GIS: 18.00, RLS-IS: 19.00
 Approximation: 1.052632, discrepancy: 1
 tmp/vgraph.metis, 100, 1258, 71, 30.0001, 23
 BEGIN_OUTPUT_FOR_TABLES
 run_type=ReduVCC
 input_graph_vertices=100
 input_graph_edges=629
 reduced_graph_vertices=71
 reduced_graph_edges=636
 total_time_to_best=0.000519276
 time_to_best_without_unwind=0.000515223
 clique_cover_size=23
 verified_cover=passed
 optimal=unknown
 
VERTEX CLIQUE COVERING PROBLEM (CCP) SOLVER
 version 0.2
 Finding clique coverings using our heuristic...
 GIS:
 Run 1:
 The algor

100%|██████████| 100/100 [00:26<00:00,  3.74it/s]


...

tmp/vgraph.metis, 100, 842, 0, 0.000154018, 34
 BEGIN_OUTPUT_FOR_TABLES
 run_type=ReduVCC
 input_graph_vertices=100
 input_graph_edges=421
 reduced_graph_vertices=0
 reduced_graph_edges=0
 total_time_to_best=5.96046e-06
 time_to_best_without_unwind=0
 clique_cover_size=34
 verified_cover=passed
 optimal=unknown
 
tmp/vgraph.metis, 100, 842, 0, 0.000148058, 34
 BEGIN_OUTPUT_FOR_TABLES
 run_type=ReduVCC
 input_graph_vertices=100
 input_graph_edges=421
 reduced_graph_vertices=0
 reduced_graph_edges=0
 total_time_to_best=5.00679e-06
 time_to_best_without_unwind=-9.53674e-07
 clique_cover_size=34
 verified_cover=passed
 optimal=unknown
 
[13:39:00] [VisCliqueDecomp] Found  34  cliques
[13:39:00]  1 / 1
Iris failed at  [29.25844266 15.11956219  2.89973861]
[13:39:03]  1 / 1
Iris failed at  [2.34784129 6.70074433 2.98677879]
[13:39:06]  1 / 1
Iris failed at  [11.58444745 10.24622144  2.53253666]
[13:39:09]  1 / 1
[13:39:18]  1 / 1
[13:39:28]  1 / 1
[13:39:37]  1 / 1
Iris failed at  [18.729302

100%|██████████| 100/100 [00:24<00:00,  4.12it/s]


VERTEX CLIQUE COVERING PROBLEM (CCP) SOLVER
 version 0.2
 Finding clique coverings using our heuristic...
 GIS:
 Run 1:
 The algorithm found a an independent set of size 17.
 IG-GCC:
 Run 53:
 Time\t\tVCC\tIS
 0.000315\t34\t30
 0.000326\t33\t30
 0.000344\t32\t30
 0.000379\t31\t30
  [iterations: 6138788][independent set: 17]
 
 The algorithm needed 18 cliques.
 
 On average, it took 0 seconds and 0 iterations. Success rate: 0 / 1.
 Final results: IG-GCC: 18.00, BRE: 0.00, GIS: 17.00, RLS-IS: 17.00
 Approximation: 1.058824, discrepancy: 1
 tmp/vgraph.metis, 100, 994, 52, 30.0001, 31
 BEGIN_OUTPUT_FOR_TABLES
 run_type=ReduVCC
 input_graph_vertices=100
 input_graph_edges=497
 reduced_graph_vertices=52
 reduced_graph_edges=364
 total_time_to_best=0.000389099
 time_to_best_without_unwind=0.000383139
 clique_cover_size=31
 verified_cover=passed
 optimal=unknown
 
VERTEX CLIQUE COVERING PROBLEM (CCP) SOLVER
 version 0.2
 Finding clique coverings using our heuristic...
 GIS:
 Run 1:
 The algori

100%|██████████| 100/100 [00:22<00:00,  4.50it/s]


tmp/vgraph.metis, 100, 742, 0, 0.000158072, 33
 BEGIN_OUTPUT_FOR_TABLES
 run_type=ReduVCC
 input_graph_vertices=100
 input_graph_edges=371
 reduced_graph_vertices=0
 reduced_graph_edges=0
 total_time_to_best=6.91414e-06
 time_to_best_without_unwind=0
 clique_cover_size=33
 verified_cover=passed
 optimal=unknown
 
tmp/vgraph.metis, 100, 742, 0, 0.000155926, 33
 BEGIN_OUTPUT_FOR_TABLES
 run_type=ReduVCC
 input_graph_vertices=100
 input_graph_edges=371
 reduced_graph_vertices=0
 reduced_graph_edges=0
 total_time_to_best=5.96046e-06
 time_to_best_without_unwind=0
 clique_cover_size=33
 verified_cover=passed
 optimal=unknown
 
[13:44:48] [VisCliqueDecomp] Found  33  cliques
[13:44:48]  1 / 1
[13:44:57]  1 / 1
[13:45:04]  1 / 1
[13:45:10]  1 / 1
[13:45:17]  1 / 1
Iris failed at  [10.2688276  10.64005811  0.01531189]
[13:45:20]  1 / 1
[13:45:29]  1 / 1
[13:45:39]  1 / 1


In [13]:
world.plot_regions(VCD.regions, opacity=0.3)

In [6]:
pts, _ = sample_cfree_handle(5,m =3123)
r,_ = iris_w_obstacles(pts,[], [])

[20:54:24]  1 / 1
[20:54:29]  1 / 1
[20:54:34]  1 / 1
[20:54:43]  1 / 1
[20:54:46]  1 / 1


In [7]:
world.plot_regions(r)

In [35]:
pts

array([[7.33879512e+00, 6.35919360e+00, 3.58044029e+00],
       [4.78985682e+00, 7.33115532e-03, 1.64176180e+00],
       [2.86847355e+00, 6.35418916e+00, 4.70033879e+00],
       [1.43195515e+00, 6.54694339e+00, 3.94139859e+00],
       [8.33889784e+00, 2.93965916e+00, 4.94516101e+00]])

ValueError: cannot reshape array of size 15 into shape (2)

In [20]:
village.plot_regions(r, opacity=0.4)

In [24]:
import numpy as np

from meshcat import Visualizer
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.transformations import translation_matrix, rotation_matrix
from matplotlib.colors import to_hex as _to_hex
from pydrake.all import HPolyhedron
import mcubes
from  pydrake.all import ( 
                          Rgba,TriangleSurfaceMesh, SurfaceTriangle,
                          MathematicalProgram, SnoptSolver, le)
import colorsys
import itertools
from fractions import Fraction
import random
from functools import partial
from meshcat.geometry import TriangularMeshGeometry
to_hex = lambda rgb: '0x' + _to_hex(rgb)[1:]
def infinite_hues():
    yield Fraction(0)
    for k in itertools.count():
        i = 2**k # zenos_dichotomy
        for j in range(1,i,2):
            yield Fraction(j,i)

def hue_to_hsvs(h: Fraction):
    # tweak values to adjust scheme
    for s in [Fraction(6,10)]:
        for v in [Fraction(6,10), Fraction(9,10)]:
            yield (h, s, v)

def rgb_to_css(rgb) -> str:
    uint8tuple = map(lambda y: int(y*255), rgb)
    return tuple(uint8tuple)

def css_to_html(css):
    return f"<text style=background-color:{css}>&nbsp;&nbsp;&nbsp;&nbsp;</text>"

def n_colors(n=33, rgbs_ret = False):
    hues = infinite_hues()
    hsvs = itertools.chain.from_iterable(hue_to_hsvs(hue) for hue in hues)
    rgbs = (colorsys.hsv_to_rgb(*hsv) for hsv in hsvs)
    csss = (rgb_to_css(rgb) for rgb in rgbs)
    to_ret = list(itertools.islice(csss, n)) if rgbs_ret else list(itertools.islice(csss, n))
    return to_ret

def n_colors_random(n=33, rgbs_ret = False):
    colors = n_colors(100 * n, rgbs_ret)
    return random.sample(colors, n)


In [34]:
#village.plot_regions(r)
self = village
def plot_regions( regions, ellipses = None,
                     region_suffix = '', colors = None,
                     wireframe = False,
                     opacity = 0.7,
                     fill = True,
                     line_width = 10,
                     darken_factor = .2,
                     el_opacity = 0.3):
    if colors is None:
        colors = n_colors_random(len(regions), rgbs_ret=True)

    for i, region in enumerate(regions):
        print(colors[i])
        c = tuple([col/255 for col in colors[i]])#,opacity)

        prefix = f"/cfree/regions{region_suffix}/{i}"
        name = prefix + "/hpoly"
        if region.ambient_dimension() == 3:
            plot_hpoly3d(name, region,
                         c, wireframe = wireframe, resolution = 30, opacity = opacity)
            
def get_AABB_limits( hpoly, dim = 3):
    max_limits = []
    min_limits = []
    A = hpoly.A()
    b = hpoly.b()

    for idx in range(dim):
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(dim, 'x')
        cost = x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A@x,b))
        solver = SnoptSolver()
        result = solver.Solve(aabbprog)
        min_limits.append(result.get_optimal_cost()-0.01)
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(dim, 'x')
        cost = -x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A@x,b))
        solver = SnoptSolver()
        result = solver.Solve(aabbprog)
        max_limits.append(-result.get_optimal_cost() + 0.01)
    return max_limits, min_limits

def get_plot_poly_mesh(region, resolution):

        def inpolycheck(q0, q1, q2, A, b):
            q = np.array([q0, q1, q2])
            res = np.min(1.0 * (A @ q - b <= 0))
            # print(res)
            return res

        aabb_max, aabb_min = get_AABB_limits(region)

        col_hand = partial(inpolycheck, A=region.A(), b=region.b())
        vertices, triangles = mcubes.marching_cubes_func(tuple(aabb_min),
                                                        tuple(aabb_max),
                                                        resolution,
                                                        resolution,
                                                        resolution,
                                                        col_hand,
                                                        0.5)
        tri_drake = [SurfaceTriangle(*t) for t in triangles]
        return vertices, triangles #, #tri_drake

def plot_hpoly3d(name, hpoly, color, wireframe = True, resolution = 30, opacity = 1.0):
    verts, triangles = get_plot_poly_mesh(hpoly,
                                            resolution=resolution)
    
    print(triangles)
    box = self[name]
    #pos = np.array(pos, dtype=float)
    col = to_hex(color)
    material = MeshLambertMaterial(color=col, opacity=opacity)
    box.set_object(TriangularMeshGeometry(verts,triangles), material)
    
    # self.set_object(name, TriangleSurfaceMesh(triangles, verts),
    #                         color, wireframe=wireframe)

In [35]:
plot_regions(r)

(61, 153, 94)
[[   2    1    0]
 [   1    4    3]
 [   0    1    3]
 ...
 [3993 3992 3410]
 [3411 3993 3410]
 [3993 3411 3406]]
(91, 229, 122)
[[   2    1    0]
 [   1    4    3]
 [   0    1    3]
 ...
 [4277 4276 4078]
 [4079 4277 4078]
 [4277 4079 4075]]
(100, 153, 61)
[[   2    1    0]
 [   1    4    3]
 [   0    1    3]
 ...
 [3273 3272 3240]
 [3241 3273 3240]
 [3273 3241 3225]]
(153, 104, 61)
[[   2    1    0]
 [   1    4    3]
 [   0    1    3]
 ...
 [3651 3650 3605]
 [3606 3651 3605]
 [3651 3606 3597]]
(118, 91, 229)
[[   2    1    0]
 [   1    4    3]
 [   0    1    3]
 ...
 [3288 3832 3292]
 [3833 3291 3293]
 [3833 3293 3292]]


logdir exists
[19:41:47] [VisSeeder] GuardInsertion attempts M: 13
[19:41:47] [VisSeeder] 0.95 probability that unseen region is less than 20.0 '%' of Cfree 


100%|██████████| 1/1 [00:00<00:00, 25266.89it/s]

[19:41:47] [VisSeeder] Found  1  hidden points
[19:41:47]  1 / 1



100%|██████████| 1/1 [00:00<00:00, 6141.00it/s]


[19:41:52] [VisSeeder] Found  1  hidden points
[19:41:52]  1 / 1


...

100%|██████████| 1/1 [00:00<00:00, 24818.37it/s]


[19:48:45] [VisSeeder] Found  1  hidden points
[19:48:45]  1 / 1
Iris failed at  [0.66016605 2.0220123  0.45910749]


100%|██████████| 1/1 [00:00<00:00, 29959.31it/s]


[19:48:51] [VisSeeder] Found  1  hidden points
[19:48:51]  1 / 1


100%|██████████| 1/1 [00:00<00:00, 30840.47it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

 42%|████▏     | 208/500 [02:03<02:53,  1.69it/s]


KeyboardInterrupt: 

In [29]:
village.plot_points(pts, radius= 0.05)

In [7]:
pts = village.sample_cfree(1000)

In [3]:
# numerical parameters
village_height = 5 
village_side = 19
building_every = 5
assert (village_side + 1) % building_every == 0

In [4]:
# helper functions for construction of village

def direction():
    I = np.eye(2)
    directions = np.vstack((I, -I))
    return directions[np.random.randint(0, 4)]

def walk(m):
    d1 = direction()
    starts = [np.zeros(2)]
    ends = [d1]
    blocks = [np.zeros(2), d1]
    for i in range(m):
        d2 = direction()
        blocks.append(blocks[-1] + d2)
        if all(d2 == d1):
            ends[-1] += d1
        else:
            starts.append(ends[-1])
            ends.append(starts[-1] + d2)
            d1 = d2
    return np.array(starts), np.array(ends), np.array(blocks)

def building(village, i, j, m):
    starts, ends, blocks = walk(m)
    offset = np.array([i, j]) + .5
    starts += offset
    ends += offset
    blocks += [i, j]
    for k, (s, e) in enumerate(zip(starts, ends)):
        c = (s + e) / 2
        d = np.abs(e - s) + 1
        r = list(.5 * d) + [village_height / 2]
        n = list(d) + [village_height]
        village.building(f'building_{i}_{j}_{k}', c, r, n)
    return blocks
        
def tree(village, i, j):
    name = f'tree_{i}_{j}'
    r = .5 # Radius of foliage.
    r_trunk = .1
    low = [i + .5, j + .5, 1]
    high = [i + .5, j + .5, village_height - .5]
    c = np.random.uniform(low, high)
    village.tree(name, c, r, r_trunk)
        
def bush(village, i, j):
    name = f'bush_{i}_{j}'
    r = np.random.uniform(low=.1, high=.35) # Radius.
    h = 4 * r # Height.
    c = np.array([i + .5, j + .5])
    village.bush(name, c, r, h)

In [5]:
village.delete()
np.random.seed(0)

# village ground
ground_color = (.7, 1, .7)
ground = village.ground(village_side, ground_color)

# buildings
blocks = []
for i in range(village_side):
    for j in range(village_side):
        if (i + 1) % building_every == 0 and (j + 1) % building_every == 0:
            blocks.append(building(village, i, j, 4))
blocks = [tuple(b) for b in np.vstack(blocks)]
    
# trees and bushes
for i in range(village_side):
    for j in range(village_side):
        if (i, j) not in blocks:
            np.random.choice([tree, bush])(village, i, j)

In [6]:
# all the obstacles are axis aligned boxes
# each row of the matrices L adn U below is a box
# the ith box is {x : L[i] <= x <= U[i]}
L = np.array(village.L)
U = np.array(village.U)
print(L.shape, U.shape)

(1780, 3) (1780, 3)
